In [1]:
elastic_rods_dir = '../../../elastic_rods/python/'
weaving_dir = '../../'
import os
import os.path as osp
import sys; sys.path.append(elastic_rods_dir); sys.path.append(weaving_dir)
import numpy as np, elastic_rods, linkage_vis
import numpy.linalg as la
from bending_validation import suppress_stdout as so
import matplotlib.pyplot as plt
from elastic_rods import EnergyType, InterleavingType

# weaving
import analysis_helper, ribbon_linkage_helper, mesh_vis, linkage_utils, compute_curve_from_curvature, pipeline_helper, importlib
importlib.reload(analysis_helper)
importlib.reload(ribbon_linkage_helper)
importlib.reload(mesh_vis)
importlib.reload(linkage_utils)
importlib.reload(compute_curve_from_curvature)
importlib.reload(pipeline_helper)
from analysis_helper import (compare_turning_angle,
                            is_on_sphere, 
                            get_distance_to_center_scalar_field, 
                            plot_curvatures, 
                            get_curvature_scalar_field,
                            construct_elastic_rod_loop_from_rod_segments, 
                            concatenate_rod_properties_from_rod_segments, 
                            compute_min_distance_rigid_transformation)
from ribbon_linkage_helper import (update_rest_curvature, 
                                   set_ribbon_linkage,
                                   export_linkage_geometry_to_obj,
                                   write_linkage_ribbon_output_florin)
from compute_curve_from_curvature import (match_geo_curvature_and_edge_len, get_all_curve_pattern)
from linkage_utils import order_segments_by_ribbons, get_turning_angle_and_length_from_ordered_rods

from pipeline_helper import (initialize_linkage, get_normal_deviation, set_joint_vector_field, stage_1_optimization, initialize_stage_2_optimizer, stage_2_optimization, InputOrganizer, write_all_output, set_surface_view_options, get_structure_analysis_view, get_double_side_view, show_selected_joints, get_fixed_boundary_joint)
import vis.fields
import matplotlib.cm as cm
import time

In [2]:
rod_length = 0.09982061776030171
width = rod_length / 20 * 5
thickness = width / 5 * 0.35
print(width, thickness)
io = InputOrganizer('ducko_v2_1', thickness, width, weaving_dir)
# io = InputOrganizer('bunny_quad_1', thickness, thickness, weaving_dir)

#  [ ear 214, ear 255, nose 39]

0.024955154440075428 0.0017468608108052797


In [3]:
OPTS = elastic_rods.NewtonOptimizerOptions()
OPTS.gradTol = 1e-6
OPTS.verbose = 1;
OPTS.beta = 1e-8
OPTS.niter = 200
OPTS.verboseNonPosDef = False
rw = 0.01
sw = 0.01

In [4]:
with so(): curved_linkage = initialize_linkage(surface_path = io.SURFACE_PATH, useCenterline = True, model_path = io.MODEL_PATH, cross_section = io.RIBBON_CS, subdivision_res = io.SUBDIVISION_RESOLUTION, interleaving_type=InterleavingType.weaving)
curved_linkage.set_design_parameter_config(use_restLen = True, use_restKappa = True)
curved_save_tgt_joint_pos = curved_linkage.jointPositions();
curved_linkage_view = get_double_side_view(curved_linkage, flip = True)
curved_linkage_view.show()

Renderer(camera=PerspectiveCamera(aspect=1.6, children=(PointLight(color='white', intensity=0.6, position=(0.0…

In [5]:
curved_linkage.setExtendedDoFsPSRL(np.load('ducko_v2_1_dof.npy'))

In [6]:
topColor =[180/255., 158/255., 119/255.]
bottomColor =[79/255., 158/255., 246/255.]
heights = curved_linkage.visualizationGeometryHeightColors()
colors = np.take(np.array([bottomColor, topColor]), heights < heights.mean(), axis=0)

In [7]:
curved_linkage.attraction_weight = 1e-5

In [8]:
def eqm_callback(prob, i):
    curved_linkage_view.update(scalarField = colors)

In [9]:
elastic_rods.compute_equilibrium(curved_linkage, callback = eqm_callback, options = OPTS, fixedVars = [])

0	0.00992099	2.88224	2.88224	1	1
1	0.00893884	12.0989	12.0989	1	1
2	0.00370837	1.8311	1.8311	1	1
3	0.0033312	0.789011	0.789011	1	1
4	0.00300203	0.554733	0.554733	1	1
5	0.00256946	0.437974	0.437974	1	1
6	0.0020248	0.334043	0.334043	1	1
7	0.00144102	0.237164	0.237164	1	1
8	0.000927836	0.158624	0.158624	1	1
9	0.000555888	0.102013	0.102013	1	1
10	0.000327332	0.0628104	0.0628104	1	1
11	0.000202906	0.0366879	0.0366879	1	1
12	0.0001361	0.0207198	0.0207198	1	1
13	9.46029e-05	0.0113857	0.0113857	1	1
14	6.45965e-05	0.00606971	0.00606971	1	1
15	4.23972e-05	0.00339548	0.00339548	1	1
16	2.72248e-05	0.00233513	0.00233513	1	1
17	1.83065e-05	0.00212008	0.00212008	1	1
18	1.4006e-05	0.00189658	0.00189658	1	1
19	1.22821e-05	0.00143698	0.00143698	1	1
20	1.16632e-05	0.00088504	0.00088504	1	1
21	1.14465e-05	0.000450479	0.000450479	1	1
22	1.13728e-05	0.000209625	0.000209625	1	1
23	1.13491e-05	8.91747e-05	8.91747e-05	1	1
24	1.13412e-05	3.54034e-05	3.54034e-05	1	1
25	1.13379e-05	1.67175e-05	1.67175e-05	1	1
26	

In [10]:
def visualize_crossing(index, linkage, view):
    vf = show_selected_joints(linkage, [index], flip = False)
    view.update(vectorField = vf, scalarField = colors)

In [14]:
ribbons = order_segments_by_ribbons(curved_linkage)
_, _, _, _, all_joint_index, _ = get_turning_angle_and_length_from_ordered_rods(ribbons, curved_linkage, rest = True)

def get_ribbon_crossing_list(index, linkage, view):
    selected_list = []
    selected_ribbon = []
    for ribbon_index, index_list in enumerate(all_joint_index):
        if index in set(index_list):
            selected_ribbon.append(ribbon_index)
            selected_list.append(index_list)
    print("The crossing {} belongs to ribbon {}".format(index, ', '.join([str(x) for x in selected_ribbon])))
    for i in range(len(selected_list)):
        print('Ribbon {}: {}'.format(selected_ribbon[i], selected_list[i]))
    return [selected_ribbon, selected_list]
    

In [15]:
def select_crossing(crossing_index):
    visualize_crossing(crossing_index, curved_linkage, curved_linkage_view)
    return get_ribbon_crossing_list(crossing_index, curved_linkage, curved_linkage_view)

vNum = [279]
[selected_ribbon, selected_list] = select_crossing(vNum[0])

vf = show_selected_joints(curved_linkage, vNum)
curved_linkage_view.update(vectorField = vf, scalarField = colors)
curved_linkage_view.show()

The crossing 279 belongs to ribbon 1, 3
Ribbon 1: [1, 34, 35, 36, 37, 38, 29, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 35, 55, 56, 33, 57, 58, 59, 60, 42, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 2, 34, 55, 88, 89, 31, 90, 91, 40, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 72, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 0, 56, 88, 120, 37, 121, 122, 123, 124, 125, 126, 127, 128, 129, 44, 130, 115, 131, 85, 132, 4, 133, 54, 36, 120, 89, 32, 57, 134, 118, 135, 86, 132, 5, 136, 137, 138, 139, 140, 141, 142, 143, 144, 106, 145, 74, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 25, 157, 158, 159, 122, 160, 53, 133, 3, 87, 135, 117, 161, 60, 41, 92, 162, 26, 157, 163, 164, 165, 166, 167, 168, 169, 62, 170, 93, 162, 27, 171, 159, 123, 172, 50, 173, 174, 175, 176, 177, 178, 108, 179, 76, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 23, 191, 1

Renderer(camera=PerspectiveCamera(aspect=1.6, children=(PointLight(color='white', intensity=0.6, position=(0.0…